In [ ]:
import os, gc, random
import numpy as np
import pandas as pd 
from pathlib import Path
import matplotlib.pyplot as plt
from typing import List, Dict
from tqdm.notebook import tqdm
from time import time, ctime

from sklearn.model_selection import KFold, GroupKFold

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import v2
from torch.optim.lr_scheduler import OneCycleLR,  CosineAnnealingWarmRestarts
from torch.optim import Adam, AdamW
from torch.cuda.amp import autocast, GradScaler

from scipy.stats import entropy
from scipy.special import rel_entr, softmax

In [ ]:
class KagglePaths:
    OUTPUT_DIR = "/kaggle/working/"
    PRE_LOADED_EEGS = '/kaggle/input/brain-eeg-spectrograms/eeg_specs.npy'
    PRE_LOADED_SPECTROGRAMS = '/kaggle/input/brain-spectrograms/specs.npy'
    TRAIN_CSV = "/kaggle/input/hms-harmful-brain-activity-classification/train.csv"
    TRAIN_EEGS = "/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/"
    TRAIN_SPECTROGRAMS = "/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/"
    TEST_CSV = "/kaggle/input/hms-harmful-brain-activity-classification/test.csv"
    TEST_SPECTROGRAMS = "/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/"
    TEST_EEGS = "/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/"

class LocalPaths:
    OUTPUT_DIR = "./outputs/"
    PRE_LOADED_EEGS = './inputs/brain-eeg-spectrograms/eeg_specs.npy'
    PRE_LOADED_SPECTROGRAMS = './inputs/brain-spectrograms/specs.npy'
    TRAIN_CSV = "./inputs/hms-harmful-brain-activity-classification/train.csv"
    TRAIN_EEGS = "./inputs/hms-harmful-brain-activity-classification/train_eegs"
    TRAIN_SPECTROGRAMS = "./inputs/hms-harmful-brain-activity-classification/train_spectrograms"
    TEST_CSV = "./inputs/hms-harmful-brain-activity-classification/test.csv"
    TEST_SPECTROGRAMS = "./inputs/hms-harmful-brain-activity-classification/test_spectrograms"
    TEST_EEGS = "./inputs/hms-harmful-brain-activity-classification/test_eegs"

PATHS = KagglePaths if os.path.exists("/kaggle") else LocalPaths

print("Output Dir: ", PATHS.OUTPUT_DIR)

EEG_FEAT_ALL = [
    'Fp1', 'F3', 'C3', 'P3', 
    'F7', 'T3', 'T5', 'O1', 
    'Fz', 'Cz', 'Pz', 'Fp2', 
    'F4', 'C4', 'P4', 'F8', 
    'T4', 'T6', 'O2', 'EKG'
    ]

EEG_FEAT_USE =  ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
EEG_FEAT_INDEX = {x:y for x,y in zip(EEG_FEAT_USE, range(len(EEG_FEAT_USE)))}

BRAIN_ACTIVITY = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']
TARGETS = [f"{lb}_vote" for lb in BRAIN_ACTIVITY]
TARGETS_PRED = [f"{lb}_pred" for lb in BRAIN_ACTIVITY]

print(EEG_FEAT_INDEX)

In [ ]:
def get_logger(log_dir, logger_name="train_model.log"):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger_file = os.path.join(log_dir, logger_name)
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=logger_file, mode="a+")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
class ModelConfig:
    SEED = 20
    SPLIT_ENTROPY = 5.5
    MODEL_NAME = "ResnetGRU_v2_Vstack_Filter_05_20_20_60_30_Normalized"
    MODEL_BACKBONE = "reset_gru"
    BATCH_SIZE = 32
    EPOCHS = 20
    EARLY_STOP_ROUNDS = 5
    GRADIENT_ACCUMULATION_STEPS = 1
    DROP_RATE = 0.15 # default: 0.1
    DROP_PATH_RATE = 0.25 # default: 0.2
    WEIGHT_DECAY = 0.01
    AMP = True
    PRINT_FREQ = 100
    NUM_WORKERS = 0 
    MAX_GRAD_NORM = 1e7
    REGULARIZATION = 0.15
    RESNET_GRU_BANDPASS = (0.5, 20) #(0.5, 20)
    RESNET_GRU_LOWPASS = 20
    RESNET_GRU_NOTCH = (60, 30) # (Freq, Quality)
    RESNET_GRU_IN_CHANNELS = 8
    RESNET_GRU_KERNELS = [3, 5, 7, 9, 11]
    RESNET_GRU_FIXED_KERNEL_SIZE = 5
    RESNET_GRU_DOWNSAMPLE = 5
    RESNET_GRU_HIDDEN_SIZE = 304 
    RESNET_GRU_DILATED = False
    

In [ ]:
seed_everything(ModelConfig.SEED)
logger = get_logger(PATHS.OUTPUT_DIR, f"{ModelConfig.MODEL_NAME}_train.log")

# Load Data

In [ ]:
def eeg_from_parquet(parquet_path: str, use_feature=EEG_FEAT_USE, display: bool = False) -> np.ndarray:
    # === Extract full length EEG Sequence ===
    # fill missing values with mean
    # first fill missing values with mean of each column
    # then if all values are missing, fill with 0
    eeg = pd.read_parquet(parquet_path, columns=use_feature)
    eeg = eeg.fillna(eeg.mean(skipna=True)).fillna(0)
    data = eeg.values.astype(np.float32)
    
    # rows = len(eeg)
    # offset = (rows - 10_000) // 2 # 50 * 200 = 10_000
    # data = data[offset:offset+10_000, :]

    if display:
        fig, ax = plt.subplots(len(use_feature), 1, figsize=(10, 2*len(use_feature)), sharex=True)
        
        for i, feat in enumerate(use_feature):
            ax[i].plot(data[:, i], label=feat)
            ax[i].legend()
            ax[i].grid()
       
        name = parquet_path.split('/')[-1].split('.')[0]
        ax[0].set_title(f'EEG {name}',size=16)
        fig.tight_layout()
        plt.show()    
    return data

In [ ]:
%%time
CREATE_EEGS = True
ALL_EEG_SIGNALS = {}
eeg_paths = list(Path(PATHS.TRAIN_EEGS).glob('*.parquet'))
preload_eegs_path = Path('./inputs/eegs_full.npy')

if CREATE_EEGS:
    count = 0
    for parquet_path in tqdm(eeg_paths, total=len(eeg_paths)):
        eeg_id = int(parquet_path.stem)
        eeg_path = str(parquet_path)
        data = eeg_from_parquet(eeg_path, display=False)
        ALL_EEG_SIGNALS[eeg_id] = data
        count += 1
    np.save("./inputs/eegs_full.npy", ALL_EEG_SIGNALS)
else:
    ALL_EEG_SIGNALS = np.load(preload_eegs_path, allow_pickle=True).item()

In [ ]:
def gen_non_overlap_samples(df_csv, targets):
    # Reference Discussion:
    # https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

    tgt_list = targets.tolist()
    brain_activity = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

    agg_dict = {
        'spectrogram_id': 'first',
        'spectrogram_label_offset_seconds': ['min', 'max'],
        'patient_id': 'first',
        'expert_consensus': 'first'
    }

    groupby = df_csv.groupby(['eeg_id'] + tgt_list)
    train = groupby.agg(agg_dict)
    train = train.reset_index()
    train.columns = ['_'.join(col).strip() for col in train.columns.values]
    train.columns = ["eeg_id"] + tgt_list + ['spectrogram_id', 'min', 'max', 'patient_id', 'target']

    # df_eeg_offsets = groupby.agg({'eeg_label_offset_seconds': lambda x: x.tolist()}).reset_index()
    # df_eeg_offsets.columns = ["eeg_id"] + tgt_list + ['eeg_offset_list']
    # df_eeg_offsets['n_segments'] = df_eeg_offsets['eeg_offset_list'].apply(len)

    # train = train.merge(df_eeg_offsets, on=['eeg_id']+tgt_list, how='left')
    
    train['total_votes'] = train[tgt_list].sum(axis=1)
    train[tgt_list] = train[tgt_list].div(train['total_votes'], axis=0)
    
    return train

In [ ]:
# Original Split 

train_csv = pd.read_csv(PATHS.TRAIN_CSV)
targets = train_csv.columns[-6:]

print("targets: ", targets.to_list())

train_csv['total_votes'] = train_csv[targets].sum(axis=1)
train_csv[targets] = train_csv[targets].astype('float32')

targets_prob = [f"{t.split('_')[0]}_prob" for t in targets]
train_csv[targets_prob] = train_csv[targets].div(train_csv['total_votes'], axis=0)

hard_csv = train_csv[train_csv['total_votes'] >= 6].copy().reset_index(drop=True)

train_all = gen_non_overlap_samples(train_csv, targets)
train_hard = gen_non_overlap_samples(hard_csv, targets)

# def get_offset_seconds(x):
#     tmp = x['eeg_offset_list']
#     if x['target'] == 'Other':
#         return [tmp[len(tmp)//2]]
#     else:
#         rand_idx = np.random.randint(0, len(tmp), len(tmp)//2)
#         return [tmp[idx] for idx in set(rand_idx)]
    
# train_all['offset_seconds'] = train_all[['target', 'eeg_offset_list']].apply(get_offset_seconds, axis=1)
# train_hard['offset_seconds'] = train_hard[['target', 'eeg_offset_list']].apply(get_offset_seconds, axis=1)

print("train_all.shape = ", train_all.shape)
print("train_all nan_count: ", train_all.isnull().sum().sum())
display(train_all.head())

print(" ")

print("train_hard.shape = ", train_hard.shape)
print("train_hard nan_count: ", train_hard.isnull().sum().sum())
display(train_hard.head())

In [ ]:
train_all[train_all['offset_seconds'].apply(len) == 0] 

# Dataset

In [ ]:
from scipy.signal import butter, lfilter, iirnotch 
from scipy.fft import fft, fftfreq

def plot_fft(ax, data, title, fs=200):
    n = len(data)
    yf = fft(data)
    xf = fftfreq(n, 1/fs)
    ax.plot(xf[:n//2], 2.0/n * np.abs(yf[0:n//2]))
    ax.set_title(title)
    ax.grid()

def bandpass_filter(data, lowcut=0.5, highcut=20, fs=200, order=2):
    low = 2 * lowcut / fs 
    high = 2 * highcut / fs
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data, axis=0)
    return y

def butter_lowpass_filter(data, cutoff_freq=20, sampling_rate=200, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data

def notch_filter(data, fs=200, f0=60, Q=30):
    b, a = iirnotch(f0, Q, fs)
    y = lfilter(b, a, data, axis=0)
    return y

def filter_eeg(eeg_seq, bandpass=(0.5, 20), lowpass=20, notch=(60, 30), fs=200):

    if bandpass is not None:
        eeg_seq = bandpass_filter(eeg_seq, lowcut=bandpass[0], highcut=bandpass[1], fs=fs, order=2)
    if lowpass is not None:
        eeg_seq = butter_lowpass_filter(eeg_seq, cutoff_freq=lowpass, sampling_rate=fs, order=4)
    if notch is not None:
        eeg_seq = notch_filter(eeg_seq, fs=fs, f0=notch[0], Q=notch[1])

    return eeg_seq

In [ ]:
class EEGSeqDataset(Dataset):
    def __init__(self, df, config, eegs, mode='train', verbose=False):
        self.df = df
        self.mode = mode
        self.eegs = eegs
        self.verbose = verbose
        self.downsample = config.RESNET_GRU_DOWNSAMPLE
        self.bandpass = config.RESNET_GRU_BANDPASS
        self.lowpass = config.RESNET_GRU_LOWPASS
        self.notch = config.RESNET_GRU_NOTCH
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        X, y_prob = self.__data_generation(idx)
        
        if self.downsample is not None:
            X = X[::self.downsample, :]
        
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y_prob, dtype=torch.float32)
    
    def __data_generation(self, index):
        row = self.df.iloc[index]
        
        if self.verbose:
            print(f"Row {index}", row[['eeg_id', 'target']].tolist())

        X = np.zeros((10_000, 8), dtype='float32')

        # if self.mode == 'train':
        #     offset_seconds = row['offset_seconds']
        #     if len(offset_seconds) == 0:
        #         offset_sec = 0
        #     else:
        #         offset_sec = int(random.choice(offset_seconds))
        # else:
        #     offset_sec = 0
        # data = self.eegs[row.eeg_id][offset_sec*200:offset_sec*200+10_000, :]
        
        data = self.eegs[row.eeg_id][:, :]

        # === Feature engineering ===
        X[:,0] = data[:,EEG_FEAT_INDEX['Fp1']] - data[:,EEG_FEAT_INDEX['T3']]
        X[:,1] = data[:,EEG_FEAT_INDEX['T3']] - data[:,EEG_FEAT_INDEX['O1']]

        X[:,2] = data[:,EEG_FEAT_INDEX['Fp1']] - data[:,EEG_FEAT_INDEX['C3']]
        X[:,3] = data[:,EEG_FEAT_INDEX['C3']] - data[:,EEG_FEAT_INDEX['O1']]

        X[:,4] = data[:,EEG_FEAT_INDEX['Fp2']] - data[:,EEG_FEAT_INDEX['C4']]
        X[:,5] = data[:,EEG_FEAT_INDEX['C4']] - data[:,EEG_FEAT_INDEX['O2']]

        X[:,6] = data[:,EEG_FEAT_INDEX['Fp2']] - data[:,EEG_FEAT_INDEX['T4']]
        X[:,7] = data[:,EEG_FEAT_INDEX['T4']] - data[:,EEG_FEAT_INDEX['O2']]

        # === Standarize ===
        X = np.clip(X, -1024, 1024)
        X = np.nan_to_num(X, nan=0) / 32.0

        # === Butter Low-pass Filter ===
        # ??? change to bandpass filter (low=0.5, hight=20, order=2) ???
        X = filter_eeg(X, bandpass=self.bandpass, lowpass=self.lowpass, notch=self.notch, fs=200)
        
        if self.mode != 'test':
            y_prob = row[TARGETS].values.astype(np.float32)
        else:
            y_prob = np.zeros(6, dtype='float32')

        return X, y_prob 

In [ ]:
# row = train_all.iloc[132]
# print(row[['eeg_id', 'min', 'max', 'target']].tolist())

# eeg_seq = ALL_EEG_SIGNALS[row.eeg_id]#[:, EEG_FEAT_INDEX['Fp1']]
# print("eeg_seq.shape: ", eeg_seq.shape)

# diff_raw = eeg_seq[:, EEG_FEAT_INDEX['Fp1']] - eeg_seq[:, EEG_FEAT_INDEX['T3']]
# diff_filt = filter_eeg(diff_raw, bandpass=(0.25, 40), lowpass=None, notch=None, fs=200)
# diff_low = filter_eeg(diff_raw, bandpass=(0.25, 40), lowpass=40, notch=None, fs=200)
# diff_notch = filter_eeg(diff_raw, bandpass=(0.25, 40), lowpass=None, notch=(60, 30), fs=200)

# fig, axes = plt.subplots(1, 4, figsize=(12, 3))
# titles = ['Raw', 'Bandpass', 'Lowpass', 'Notch']
# for i, data in enumerate([diff_raw, diff_filt, diff_low, diff_notch]):
#     plot_fft(axes[i], data, titles[i])
#     axes[i].grid(True)
# fig.tight_layout()
# plt.show()


# fig, ax = plt.subplots(1, 1, figsize=(12, 3))
# x_slice = np.arange(2000, 4000, 5)
# ax.plot(diff_raw[x_slice], label='Raw')
# ax.plot(diff_filt[x_slice], label='Bandpass')
# ax.plot(diff_low[x_slice], label='Lowpass')
# ax.plot(diff_notch[x_slice], label='Notch')
# ax.legend()
# fig.tight_layout()
# plt.show()


In [ ]:
# visualize the dataset
train_dataset = EEGSeqDataset(train_all, ModelConfig, ALL_EEG_SIGNALS, mode="train", verbose=False)
train_loader = DataLoader(train_dataset, drop_last=True, batch_size=16, num_workers=4, pin_memory=True, shuffle=False)

for batch in train_loader:
    X, y = batch
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    
    fig, axes = plt.subplots(4, 1, figsize=(20, 20))
    ax_idx = 0
    for item in range(4): #np.random.choice(range(X.shape[0]), 4):
        offset = 0
        for col in range(X.shape[-1]):
            if col != 0:
                offset -= X[item,:,col].min()
            axes[ax_idx].plot(np.arange(X.shape[1]), X[item,:,col]+offset, label=f'feature {col+1}')
            offset += X[item,:,col].max()
        print(y[item])
        # axes[ax_idx].set_title(f'Weight = {weights[item]}',size=14)
        axes[ax_idx].legend()
        ax_idx += 1
    fig.tight_layout()
    plt.show()
    break

del train_dataset, train_loader
torch.cuda.empty_cache()
gc.collect()

# Model

In [ ]:
class ResNet_1D_Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, downsampling, dropout=0.0, dilation=1):
        super(ResNet_1D_Block, self).__init__()
        self.block = nn.Sequential(
            nn.BatchNorm1d(num_features=in_channels),
            nn.Hardswish(), #nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(num_features=out_channels),
            nn.Hardswish(), #nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Conv1d(out_channels, out_channels, kernel_size, stride, padding, dilation=dilation, bias=False),
        )
        
        self.downsampling = downsampling

    def forward(self, x):
        identity = x
        out = self.block(x)
        
        if self.downsampling is not None:
            identity = self.downsampling(identity)
            out = self.downsampling(out)
            
        out += identity
        return out

class SelfAttentionPooling(nn.Module):
    """
    Implementation of SelfAttentionPooling 
    Original Paper: Self-Attention Encoding and Pooling for Speaker Recognition
    https://arxiv.org/pdf/2008.01077v1.pdf
    """
    def __init__(self, input_dim):
        super(SelfAttentionPooling, self).__init__()
        self.W = nn.Linear(input_dim, 1)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, batch_rep):
        """
        input:
            batch_rep : size (N, T, H), N: batch size, T: sequence length, H: Hidden dimension
        attention_weight:
            att_w : size (N, T, 1)
        return:
            utter_rep: size (N, H)
        """
        att_w = self.softmax(self.W(batch_rep).squeeze(-1)).unsqueeze(-1)
        utter_rep = torch.sum(batch_rep * att_w, dim=1)

        return utter_rep

class ResNetGRU(nn.Module):
    def __init__(self, config=ModelConfig, num_classes=6):
        super(ResNetGRU, self).__init__()

        self.planes = 24
        self.kernels = config.RESNET_GRU_KERNELS
        self.in_channels = config.RESNET_GRU_IN_CHANNELS
        self.use_dilation = config.RESNET_GRU_DILATED

        self.conv_in_size = self.planes * len(self.kernels)

        fixed_kernel_size = config.RESNET_GRU_FIXED_KERNEL_SIZE
        rnn_hidden_size = 128
        hidden_size = self.conv_in_size + rnn_hidden_size*2 #config.RESNET_GRU_HIDDEN_SIZE

        # Define the separate convolutional layers
        self.parallel_conv = self._make_parallel_conv_layers()
        # Define the ResNet part of the model
        self.resnet_part = self._make_resnet_part(fixed_kernel_size, n_blocks=9)
        # Define the GRU part of the model
        self.rnn = nn.GRU(input_size=self.in_channels, hidden_size=rnn_hidden_size, num_layers=1, bidirectional=True)
        
        # Define the pooling layers
        self.pooling_resnet = SelfAttentionPooling(self.conv_in_size)
        self.pooling_rnn = SelfAttentionPooling(rnn_hidden_size * 2)
        
        # Define the final fully connected layer
        self.fc = nn.Linear(in_features=hidden_size, out_features=num_classes)

    def _make_parallel_conv_layers(self):
        
        return nn.ModuleList([
            nn.Conv1d(
                in_channels=self.in_channels, 
                out_channels=self.planes, 
                kernel_size=kernel_size,
                stride=1, 
                padding=kernel_size//2, 
                bias=False
            ) for kernel_size in self.kernels
        ])

    def _make_resnet_part(self, fixed_kernel_size, n_blocks=9):
        # prepare resnet layers
        if self.use_dilation:
            dilation_rates = [1, 2, 2, 2, 2, 4, 4, 4, 4] #[1] * n_blocks
        else:
            dilation_rates = [1] * n_blocks
        paddings = [fixed_kernel_size//2 * rate for rate in dilation_rates]
        
        ds_layer = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        ds_layers = [ds_layer if i % 2 == 0 else None for i in range(n_blocks-1)] + [None]
        
        resnet_layers = [
            ResNet_1D_Block(
                in_channels=self.conv_in_size,
                out_channels=self.conv_in_size, 
                kernel_size=fixed_kernel_size, 
                stride=1, 
                padding=paddings[i], 
                downsampling=ds_layers[i],
                dropout=0.0,
                dilation=dilation_rates[i])
            for i in range(n_blocks)
        ]
        
        # return the resnet encoder
        return nn.Sequential(
            nn.BatchNorm1d(num_features=self.conv_in_size),
            nn.SiLU(), #nn.ReLU(inplace=False),
            nn.Conv1d( # -> downsample by stride=2
                in_channels=self.conv_in_size, 
                out_channels=self.conv_in_size,
                kernel_size=fixed_kernel_size, 
                stride=1, 
                padding=fixed_kernel_size//2, 
                bias=False
            ), 
            *resnet_layers,
            nn.BatchNorm1d(num_features=self.conv_in_size),
            nn.SiLU(), #nn.ReLU(inplace=False),
            nn.AvgPool1d(kernel_size=5, stride=5, padding=0)
        )
    
    def forward(self, x):
        
        x = x.permute(0, 2, 1)

        cov_sep = [conv(x) for conv in self.parallel_conv]
        cov_out = torch.cat(cov_sep, dim=1) # Vstack -> (N, sum(C_i), L)
        
        # - pass through the resnet part
        resnet_out = self.resnet_part(cov_out)
        out_1 = self.pooling_resnet(resnet_out.permute(0, 2, 1))
        
        # - extract features using rnn
        rnn_out, _ = self.rnn(x.permute(0, 2, 1))
        out_2 = self.pooling_rnn(rnn_out)

        # - concat features and pass to FC
        feat = torch.cat([out_1, out_2], dim=1) 
        result = self.fc(feat)  
        
        return result


In [ ]:
train_dataset = EEGSeqDataset(train_all, ModelConfig, ALL_EEG_SIGNALS, mode="train")
train_loader = DataLoader(train_dataset, drop_last=True, batch_size=4, num_workers=4, pin_memory=True, shuffle=False)

model = ResNetGRU(config=ModelConfig, num_classes=6)

for i, batch in enumerate(train_loader):
    X, y = batch

    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    
    y_pred = model(X)
    print(y_pred.shape)
    break 

del model, train_dataset, train_loader, X, y
gc.collect()

In [ ]:
!nvidia-smi

# Train

In [ ]:
import warnings
warnings.filterwarnings("ignore")

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
class Trainer:

    def __init__(self, model, config, logger):

        self.model = model
        self.logger = logger
        self.config = config
        
        self.early_stop_rounds = config.EARLY_STOP_ROUNDS
        self.early_stop_counter = 0
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.kl_div_loss = nn.KLDivLoss(reduction='batchmean')
        self.ce_loss = nn.CrossEntropyLoss()
        self.gamma = config.REGULARIZATION
        
        # self.criterion = nn.KLDivLoss(reduction="batchmean")
    
    def criterion(self, y_pred, y_true, weights=None, mode='train'):
        kl_loss = self.kl_div_loss(F.log_softmax(y_pred, dim=1), y_true)
        if (self.gamma is not None) & (mode == 'train'):
            softmax_probs = F.softmax(y_pred, dim=1)  # Compute softmax probabilities
            entropy_loss = -(softmax_probs * torch.log(softmax_probs + 1e-9)).sum(dim=1).mean(dim=0) # Compute entropy, add epsilon to avoid log(0)
            return kl_loss - self.gamma * entropy_loss
        else:
            return kl_loss
        
    def train(self, train_loader, valid_loader, from_checkpoint=None):

        self.optimizer = AdamW(self.model.parameters(), lr=8e-3, weight_decay=self.config.WEIGHT_DECAY)

        # CosineAnnealingWarmRestarts( 
        #     self.optimizer,
        #     T_0=20,
        #     eta_min=1e-6,
        #     T_mult=1,
        #     last_epoch=-1
        # )
        self.scheduler =  OneCycleLR(
            self.optimizer,
            max_lr=1e-4,
            epochs=self.config.EPOCHS,
            steps_per_epoch=len(train_loader),
            pct_start=0.1,
            anneal_strategy="cos",
            final_div_factor=100,
        )

        if from_checkpoint is not None:
            self.model.load_state_dict(torch.load(from_checkpoint, map_location=self.device))

        self.model.to(self.device)
        best_weights, best_preds, best_loss = None, None, float("inf")
        loss_records = {"train": [], "valid": []}

        for epoch in range(self.config.EPOCHS):
            start_epoch = time()

            train_loss, _ = self._train_or_valid_epoch(epoch, train_loader, is_train=True)
            valid_loss, valid_preds = self._train_or_valid_epoch(epoch, valid_loader, is_train=False)

            loss_records["train"].append(train_loss)
            loss_records["valid"].append(valid_loss)

            elapsed = time() - start_epoch

            info = f"{'-' * 100}\nEpoch {epoch + 1} - "
            info += f"Average Loss: (train) {train_loss:.4f}; (valid) {valid_loss:.4f} | Time: {elapsed:.2f}s"
            self.logger.info(info)

            if valid_loss < best_loss:
                best_loss = valid_loss
                best_weights = self.model.state_dict()
                best_preds = valid_preds
                self.logger.info(f"Best model found in epoch {epoch + 1} | valid loss: {best_loss:.4f}")
                self.early_stop_counter = 0
            
            else:
                self.early_stop_counter += 1
                if self.early_stop_counter >= self.early_stop_rounds:
                    self.logger.info(f"Early stopping at epoch {epoch + 1}")
                    break

        return best_weights, best_preds, loss_records

    def _train_or_valid_epoch(self, epoch_id, dataloader, is_train=True):

        self.model.train() if is_train else self.model.eval()
        mode = "Train" if is_train else "Valid"

        len_loader = len(dataloader)
        scaler = GradScaler(enabled=self.config.AMP)
        loss_meter, predicts_record = AverageMeter(), []

        start = time()
        pbar = tqdm(dataloader, total=len(dataloader), unit="batch", desc=f"{mode} [{epoch_id}]")
        for step, (X, y) in enumerate(pbar):
            X, y = X.to(self.device), y.to(self.device)

            if is_train:
                with autocast(enabled=self.config.AMP):
                    y_pred = self.model(X)
                    loss = self.criterion(y_pred, y)
                if self.config.GRADIENT_ACCUMULATION_STEPS > 1:
                    loss = loss / self.config.GRADIENT_ACCUMULATION_STEPS
                scaler.scale(loss).backward()
                grad_norm = torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.MAX_GRAD_NORM)
                if (step + 1) % self.config.GRADIENT_ACCUMULATION_STEPS == 0:
                    scaler.step(self.optimizer)
                    scaler.update()
                    self.optimizer.zero_grad()
                    self.scheduler.step()
            else:
                with torch.no_grad():
                    y_pred = self.model(X)
                    loss = self.criterion(y_pred, y, mode='valid')
                if self.config.GRADIENT_ACCUMULATION_STEPS > 1:
                    loss = loss / self.config.GRADIENT_ACCUMULATION_STEPS
                
                predicts_record.append(y_pred.to('cpu').numpy())
            
            loss_meter.update(loss.item(), y.size(0))
            end = time()

            if (step % self.config.PRINT_FREQ == 0) or (step == (len_loader - 1)):
                lr = self.scheduler.get_last_lr()[0]
                info = f"Epoch {epoch_id + 1} [{step}/{len_loader}] | {mode} Loss: {loss_meter.avg:.4f}"
                if is_train:
                    info += f" Grad: {grad_norm:.4f} LR: {lr:.4e}"
                info += f" | Elapse: {end - start:.2f}s"
                print(info)

        if not is_train:
            predicts_record = np.concatenate(predicts_record)
            
        return loss_meter.avg, predicts_record


In [ ]:
def train_fold(model, fold_id, train_folds, valid_folds, logger, stage=1, checkpoint=None):

    train_dataset = EEGSeqDataset(train_folds, ModelConfig, ALL_EEG_SIGNALS, mode="train")
    valid_dataset = EEGSeqDataset(valid_folds, ModelConfig, ALL_EEG_SIGNALS, mode="valid")

    # ======== DATALOADERS ==========
    loader_kwargs = {
        "batch_size": ModelConfig.BATCH_SIZE,
        "num_workers": ModelConfig.NUM_WORKERS,
        "pin_memory": True,
        "shuffle": False,
    }

    train_loader = DataLoader(train_dataset, drop_last=True, collate_fn=None, **loader_kwargs)
    valid_loader = DataLoader(valid_dataset, drop_last=False, collate_fn=None, **loader_kwargs)

    if checkpoint is not None:
        print(f"Loading model from checkpoint: {checkpoint}")

    trainer = Trainer(model, ModelConfig, logger)
    best_weights, best_preds, loss_records = trainer.train(
        train_loader, valid_loader, from_checkpoint=checkpoint)

    save_model_name = f"{ModelConfig.MODEL_NAME}_fold_{fold_id}_stage_{stage}.pth"
    torch.save(best_weights, os.path.join(PATHS.OUTPUT_DIR, save_model_name))

    del train_dataset, valid_dataset, train_loader, valid_loader
    torch.cuda.empty_cache()
    gc.collect()

    return best_preds, loss_records

def evaluate_oof(oof_df):
    '''
    Evaluate the out-of-fold dataframe using KL Divergence (torch and kaggle)
    '''
    kl_loss = nn.KLDivLoss(reduction="batchmean")
    labels = torch.tensor(oof_df[TARGETS].values.astype('float32'))
    preds = F.log_softmax(
        torch.tensor(oof_df[TARGETS_PRED].values.astype('float32'), requires_grad=False),
        dim=1
    )
    kl_torch = kl_loss(preds, labels).item()

    return kl_torch

def prepare_k_fold(df, k_folds=5):

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=ModelConfig.SEED)
    unique_spec_id = df['spectrogram_id'].unique()
    df['fold'] = k_folds

    for fold, (train_index, valid_index) in enumerate(kf.split(unique_spec_id)):
        df.loc[df['spectrogram_id'].isin(unique_spec_id[valid_index]), 'fold'] = fold

    return df

In [22]:
from kl_divergence import score as kaggle_score 
from sklearn.metrics import confusion_matrix
import seaborn as sns

TARGET2ID = {'Seizure': 0, 'LPD': 1, 'GPD': 2, 'LRDA': 3, 'GRDA': 4, 'Other': 5}

def calc_kaggle_score(oof_df):
    submission_df = oof_df[['eeg_id']+TARGETS_PRED].copy()
    submission_df.columns = ['eeg_id'] + TARGETS
    solution_df = oof_df[['eeg_id']+TARGETS].copy()
    return kaggle_score(solution_df, submission_df, 'eeg_id')

def analyze_oof(oof_csv):

    kl_criteria = nn.KLDivLoss(reduction='batchmean')
    softmax = nn.Softmax(dim=1)

    oof_df = pd.read_csv(oof_csv)
    oof_df['target_pred'] = oof_df[TARGETS_PRED].apply(lambda x: np.argmax(x), axis=1)
    oof_df['target_id'] = oof_df[TARGETS].apply(lambda x: np.argmax(x), axis=1)
    
    oof_df["kl_loss"] = oof_df.apply(
    lambda row: 
        kl_criteria(
            F.log_softmax(
                    torch.tensor(row[TARGETS_PRED].values.astype(np.float32)).unsqueeze(0)
                , dim=1
                ), 
            torch.tensor(row[TARGETS].values.astype(np.float32))
            ).numpy(),
    axis=1)

    oof_df["kl_loss"] = oof_df['kl_loss'].astype(np.float32)

    oof_df[TARGETS_PRED] = softmax( torch.tensor(oof_df[TARGETS_PRED].values.astype(np.float32)) )

    oof_df.head()

    return oof_df

In [24]:
# Major Train Loop
# ================== Logger ==================
logger.info(f"{'*' * 100}")
logger.info(f"Script Start: {ctime()}")
logger.info(f"Model Configurations:")
for key, value in ModelConfig.__dict__.items():
    if not key.startswith("__"):
        logger.info(f"{key}: {value}")
logger.info(f"{'*' * 100}")

# ================== Prepare Training ==================
oof_stage_1, oof_stage_2 = pd.DataFrame(), pd.DataFrame()
loss_history_1, loss_history_2 = [], []
t_start = time()

K_FOLDS = 5
train_all = prepare_k_fold(train_all, k_folds=K_FOLDS)

for fold in range(0, K_FOLDS):
    tik_total = time()
    

    valid_folds = train_all[(train_all['fold'] == fold) ].reset_index(drop=True)
    train_folds = train_all[(train_all['fold'] != fold) ].reset_index(drop=True)
    train_size, valid_size = train_folds.shape[0], valid_folds.shape[0]

    # ================== Stage 1: Train ====================
    tik = time()
    ## STAGE 1
    model = ResNetGRU(config=ModelConfig, num_classes=6)

    logger.info(f"{'=' * 100}\nFold: {fold}\n{'=' * 100}")
    logger.info(f"- Stage 1 | Train: {train_size}; Valid: {valid_size} -")
    valid_predicts, loss_records = train_fold(
        model, fold, train_folds, valid_folds, logger, stage=1, checkpoint=None)

    loss_history_1.append(loss_records)
    valid_folds[TARGETS_PRED] = valid_predicts
    kl_loss_torch = evaluate_oof(valid_folds)
    info = f"{'=' * 100}\nFold {fold} Valid Loss: {kl_loss_torch}\n"
    info += f"Elapse: {(time() - tik) / 60:.2f} min \n{'=' * 100}"
    logger.info(info)

    oof_stage_1 = pd.concat([oof_stage_1, valid_folds], axis=0).reset_index(drop=True)
    oof_stage_1.to_csv(os.path.join(PATHS.OUTPUT_DIR, f"{ModelConfig.MODEL_NAME}_oof_1.csv"), index=False)

    # ================== Stage 2: Train ====================
    tik = time()

    ModelConfig.REGULARIZATION = 0.2 

    model = ResNetGRU(config=ModelConfig, num_classes=6)
    
    train_folds_2 = train_hard[~train_hard['eeg_id'].isin(valid_folds['eeg_id'])].reset_index(drop=True)
    valid_folds_2 = train_hard[ train_hard['eeg_id'].isin(valid_folds['eeg_id'])].reset_index(drop=True)
    train_size = train_folds_2.shape[0]
    valid_size = valid_folds_2.shape[0]
    
    ## STAGE 2
    logger.info(f"- Stage 2 | Train: {train_size}; Valid: {valid_size} | Reg = {ModelConfig.REGULARIZATION} -")

    # model_dir = "/home/shiyi/kaggle_hms/outputs/ResnetGRU_Originalsplit/Reg015"
    # checkpoint = list(Path(model_dir).glob(f"*_fold_{fold}_stage_1.pth"))[0]
    checkpoint = list(Path(PATHS.OUTPUT_DIR).glob(f"{ModelConfig.MODEL_NAME}_fold_{fold}_stage_1.pth"))[0]

    valid_predicts, loss_records = train_fold(
        model, fold, train_folds_2, valid_folds_2, logger, stage=2, checkpoint=checkpoint)
    
    loss_history_2.append(loss_records)
    valid_folds_2[TARGETS_PRED] = valid_predicts
    kl_loss_torch = evaluate_oof(valid_folds_2)
    info = f"{'=' * 100}\nFold {fold} Valid Loss: {kl_loss_torch}\n"
    info += f"Elapse: {(time() - tik) / 60:.2f} min \n{'=' * 100}"
    logger.info(info)

    oof_stage_2 = pd.concat([oof_stage_2, valid_folds_2], axis=0).reset_index(drop=True)
    oof_stage_2.to_csv(os.path.join(PATHS.OUTPUT_DIR, f"{ModelConfig.MODEL_NAME}_oof_2.csv"), index=False)

    logger.info(f"Fold {fold} Elapse: {(time() - tik_total) / 60:.2f} min")

info = f"{'=' * 100}\nTraining Complete!\n"
cv_results_1 = evaluate_oof(oof_stage_1)
cv_results_2 = evaluate_oof(oof_stage_2)
info += f"CV Result: Stage 1: {cv_results_1} | Stage 2: {cv_results_2}\n"
info += f"Elapse: {(time() - t_start) / 60:.2f} min \n{'=' * 100}"
logger.info(info)

****************************************************************************************************
Script Start: Sat Apr  6 23:44:12 2024
Model Configurations:
SEED: 20
SPLIT_ENTROPY: 5.5
MODEL_NAME: ResnetGRU_v2_Vstack_Filter_05_20_20_60_30_Normalized
MODEL_BACKBONE: reset_gru
BATCH_SIZE: 32
EPOCHS: 20
EARLY_STOP_ROUNDS: 5
GRADIENT_ACCUMULATION_STEPS: 1
DROP_RATE: 0.15
DROP_PATH_RATE: 0.25
WEIGHT_DECAY: 0.01
AMP: True
PRINT_FREQ: 100
NUM_WORKERS: 0
MAX_GRAD_NORM: 10000000.0
REGULARIZATION: None
RESNET_GRU_BANDPASS: (0.5, 20)
RESNET_GRU_LOWPASS: 20
RESNET_GRU_NOTCH: (60, 30)
RESNET_GRU_IN_CHANNELS: 8
RESNET_GRU_KERNELS: [3, 5, 7, 9, 11]
RESNET_GRU_FIXED_KERNEL_SIZE: 5
RESNET_GRU_DOWNSAMPLE: 5
RESNET_GRU_HIDDEN_SIZE: 304
RESNET_GRU_DILATED: False
****************************************************************************************************
- Stage 2 | Train: 5215; Valid: 1277 | Reg = 0.2 -


Loading model from checkpoint: outputs/ResnetGRU_v2_Vstack_Filter_05_20_20_60_30_Normalized_fold_0_stage_1.pth


Train [0]:   0%|          | 0/162 [00:00<?, ?batch/s]

Epoch 1 [0/162] | Train Loss: 0.2301 Grad: 1341501.8750 LR: 4.0023e-06 | Elapse: 0.11s
Epoch 1 [100/162] | Train Loss: 0.1275 Grad: 417967.6875 LR: 2.5357e-05 | Elapse: 10.03s
Epoch 1 [161/162] | Train Loss: 0.1129 Grad: 208554.8281 LR: 5.2233e-05 | Elapse: 16.06s


Valid [0]:   0%|          | 0/40 [00:00<?, ?batch/s]

...

Epoch 17 [0/42] | Valid Loss: 0.5143 | Elapse: 0.10s


----------------------------------------------------------------------------------------------------
Epoch 17 - Average Loss: (train) -0.1034; (valid) 0.4009 | Time: 19.58s
Early stopping at epoch 17
Fold 4 Valid Loss: 0.39711377024650574
Elapse: 5.59 min 


Epoch 17 [41/42] | Valid Loss: 0.4009 | Elapse: 3.81s


Fold 4 Elapse: 5.59 min


KeyError: "None of [Index(['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote',\n       'other_vote'],\n      dtype='object')] are in the [columns]"

In [25]:
# nfo = f"{'=' * 100}\nTraining Complete!\n"
# cv_results_1 = None 
# #evaluate_oof(oof_stage_1)
# cv_results_2 = evaluate_oof(oof_stage_2)
# info += f"CV Result: Stage 1: {cv_results_1} | Stage 2: {cv_results_2}\n"
# info += f"Elapse: {(time() - t_start) / 60:.2f} min \n{'=' * 100}"
# logger.info(info)

Training Complete!
CV Result: Stage 1: None | Stage 2: 0.40033119916915894
Elapse: 20.79 min 


In [ ]:
# plot loss history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=False)

for i, loss in enumerate(loss_history_1):
    # ax1.plot(loss['train'], marker="*", ls="-", label=f"Fold {i} Train")
    ax1.plot(loss['valid'], marker="o", ls=":", label=f"Fold {i} Valid")

for i, loss in enumerate(loss_history_2):
    # ax2.plot(loss['train'], marker="*", ls="-", label=f"Fold {i} Train")
    ax2.plot(loss['valid'], marker="o", ls=":", label=f"Fold {i} Valid")

ax1.set_title("Stage 1 Loss")
ax2.set_title("Stage 2 Loss")

for ax in (ax1, ax2):
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.legend()
    ax.grid(True)

fig.tight_layout()
fig.savefig(Path(PATHS.OUTPUT_DIR) / f"{ModelConfig.MODEL_NAME}_loss_history.png")
plt.show()

In [ ]:
csv_path = f'./outputs/{ModelConfig.MODEL_NAME}_oof_1.csv'
print("CSV Path: ", csv_path)

oof_df = analyze_oof(csv_path)

print("Kaggle Score: ", calc_kaggle_score(oof_df))
print("Average KL Loss: ", oof_df["kl_loss"].mean())

display(oof_df.head())

# plot confusion matrix
cm = confusion_matrix(oof_df['target_id'], oof_df['target_pred']) # (y_true, y_pred)
cm = cm / cm.sum(axis=1)[:, np.newaxis]

fig = plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=TARGET2ID.keys(), yticklabels=TARGET2ID.keys())
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.title(csv_path.split('/')[-1].split('.')[0], fontsize=12)
fig.tight_layout()
fig.savefig(f"./outputs/{csv_path.split('/')[-1].split('.')[0]}_CM.png")
plt.show()

In [ ]:
csv_path = f'./outputs/{ModelConfig.MODEL_NAME}_oof_2.csv'
print("CSV Path: ", csv_path)

oof_df = analyze_oof(csv_path)

print("Kaggle Score: ", calc_kaggle_score(oof_df))
print("Average KL Loss: ", oof_df["kl_loss"].mean())

display(oof_df.head())

# plot confusion matrix
cm = confusion_matrix(oof_df['target_id'], oof_df['target_pred']) # (y_true, y_pred)
cm = cm / cm.sum(axis=1)[:, np.newaxis]

fig = plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=TARGET2ID.keys(), yticklabels=TARGET2ID.keys())
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.title(csv_path.split('/')[-1].split('.')[0], fontsize=12)
fig.tight_layout()
fig.savefig(f"./outputs/{csv_path.split('/')[-1].split('.')[0]}_CM.png")
plt.show()

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(15, 15), sharex=True, sharey=True)

oof_samples = oof_df.loc[0:len(oof_df):250]

for i, ax in enumerate(axes.flatten()):
    row = oof_samples.iloc[i]
    x = np.arange(6)
    ax.plot(x, row[TARGETS].T, marker="o", ls="-", label="True")
    ax.plot(x, row[TARGETS_PRED].T, marker="*", ls="--", label="Predicted")
    ax.set_title(f"{row['target']} | KL Loss: {row['kl_loss']:.4f}")
    ax.legend()
    
fig.tight_layout()
fig.savefig(f"./outputs/{csv_path.split('/')[-1].split('.')[0]}_samples.png")
plt.show()